In [7]:
# pip install tensorflow==2.15.1 

In [8]:
import keras
print(keras.__version__)

2.15.0


In [3]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Bidirectional, LSTM, Concatenate, Dropout, Dense, Layer, Multiply, TimeDistributed, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from keras_tuner import HyperModel, RandomSearch, BayesianOptimization, Hyperband
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorboard.plugins.hparams import api as hp
from tensorflow.keras import models, layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer

In [4]:
url_data = 'https://raw.githubusercontent.com/TeodorRusKvi/Tekstanalyse/main/git_NLP_data/'

df = pd.read_csv(url_data + 'new_df.csv')

df['All_text'] = df['All_text'].replace(['U.S.', 'U.S.A.'], ['US', 'USA'], regex=True)
df['Processed'] = df['Processed'].fillna(0)
df['Processed'] = df['Processed'].astype(str)
df['All_text'] = df['All_text'].fillna(0)
df['All_text'] = df['All_text'].astype(str)

# df.to_csv('new_df.csv', index=False)

# Making the relevant columns to lists
all_texts = (df['All_text'].to_list())
texts = df['Processed'].to_list()

# Setting the wanted text for further modelling
corpus = texts

tokenizer = Tokenizer(oov_token='<OOV>') # Hyperparameters = num_words=vocab_size, oov_token=oov_tok
tokenizer.fit_on_texts(corpus)

sequences = tokenizer.texts_to_sequences(corpus)

word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.\n\nFirst 10 is listen below:')
print(dict(list(word_index.items())[0:10]))

Found 22234 unique tokens.

First 10 is listen below:
{'<OOV>': 1, 'people': 2, 'like': 3, 'work': 4, 'right': 5, 'trump': 6, 'think': 7, 'state': 8, 'government': 9, 'party': 10}


In [5]:
df

,All_text,Political_Lean,Subreddit,subreddit_encoded,Processed,Dependency_Tags,POS_Tags,Named_Entities
0,"No matter who someone is, how they look like, ...",Liberal,socialism,14,matter look like language speak wear remember ...,"['advmod', 'ROOT', 'prep', 'compound', 'compou...","['ADV', 'VERB', 'ADP', 'NOUN', 'NOUN', 'NOUN',...",[]
1,Biden speech draws 38.2 million US TV viewers 0,Liberal,democrats,10,biden speech draw million U.S. tv viewer,"['compound', 'nsubj', 'ROOT', 'nummod', 'compo...","['PROPN', 'NOUN', 'VERB', 'NUM', 'PROPN', 'NOU...","['biden speech draw million (ORG)', 'U.S. (GPE)']"
2,State of the union Who watched the state of th...,Liberal,DemocraticSocialism,2,state union watch state union night opinion,"['compound', 'nsubj', 'ROOT', 'compound', 'com...","['NOUN', 'PROPN', 'VERB', 'NOUN', 'PROPN', 'NO...",['state union watch state union (ORG)']
3,We Should Just Give Poor People Money 0,Liberal,SocialDemocracy,6,poor people money,"['amod', 'compound', 'ROOT']","['ADJ', 'NOUN', 'NOUN']",[]
4,Do it for the Dew 0,Liberal,democrats,10,dew,['ROOT'],['NOUN'],[]
...,...,...,...,...,...,...,...,...
12849,Ron Paul’s Spirited Defense of WikiLeaks & Fre...,Conservative,anarchocapitalism,8,ron paul spirited defense wikileaks free infor...,"['compound', 'nsubj', 'amod', 'compound', 'nsu...","['PROPN', 'PROPN', 'VERB', 'NOUN', 'NOUN', 'AD...",['ron paul (PERSON)']
12850,"“Anarcho-capitalism, in my opinion, is a doctr...",Conservative,anarchocapitalism,8,anarcho capitalism opinion doctrinal system im...,"['nmod', 'compound', 'compound', 'amod', 'nsub...","['PROPN', 'PROPN', 'NOUN', 'ADJ', 'NOUN', 'VER...",[]
12851,Mises Wiki is a wiki project dedicated to the ...,Conservative,anarchocapitalism,8,mises wiki wiki project dedicate advancement a...,"['ROOT', 'compound', 'compound', 'nmod', 'amod...","['VERB', 'ADJ', 'NOUN', 'NOUN', 'NOUN', 'NOUN'...","['mises wiki wiki (PERSON)', 'austrian (NORP)'..."
12852,Fireman Protection Monopoly - Is This Failed C...,Conservative,anarchocapitalism,8,fireman protection monopoly failed capitalism,"['compound', 'compound', 'nsubj', 'ROOT', 'dobj']","['PROPN', 'PROPN', 'NOUN', 'VERB', 'NOUN']",['fireman protection monopoly (ORG)']


In [12]:
# Last inn 'X_train_LSTM' fra en CSV-fil
X_train_LSTM = pd.read_csv(url_data+'X_tensorflow.csv')
# Konverter hele DataFrame til et NumPy array
X_train_LSTM = X_train_LSTM.to_numpy()

# Last inn 'y_train_LSTM' fra en CSV-fil
y_train_df = pd.read_csv(url_data+'y_liberal.csv')
# Konverter hele DataFrame til et NumPy array
y_train_LSTM = y_train_df.to_numpy()

# Last inn 'y_train_LSTM' fra en CSV-fil
embeddings_GloVe = pd.read_csv(url_data+'embeddings_glove.csv')
# Konverter hele DataFrame til et NumPy array
embeddings_GloVe = embeddings_GloVe.to_numpy()


# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_train_LSTM, y_train_LSTM, test_size=0.3, random_state=42)

In [13]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

from tensorboard import notebook
notebook.list() # View open TensorBoard instances

Known TensorBoard instances:
  - port 6008: logdir logs/fit (started 5 days, 14:01:39 ago; pid 1956)
  - port 6006: logdir logs/hparam_tuning (started 9 days, 14:48:52 ago; pid 23884)


In [14]:
# Define hyperparameters with TensorBoard HParams API
HP_FILTERS = hp.HParam('filters', hp.Discrete([32, 35]))
HP_NUM_UNITS2 = hp.HParam('num_units2', hp.Discrete([64, 128, 152]))
HP_DROPOUT2 = hp.HParam('dropout2', hp.RealInterval(0.2, 0.6))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.RealInterval(0.001, 0.01))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd', 'rmsprop']))

In [15]:
%tensorboard --logdir=logs/fit

Reusing TensorBoard on port 6008 (pid 1956), started 5 days, 14:01:39 ago. (Use '!kill 1956' to kill it.)

In [16]:
from keras.layers import Layer
import keras.backend as K
from tensorflow.keras.layers import AdditiveAttention, Attention


class Attention(Layer): #BahdanauAttention
    def __init__(self,**kwargs):
        super(Attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(Attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(Attention,self).get_config()


class BahdanauAttention(Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = Dense(units)
        self.W2 = Dense(units)
        self.V = Dense(1)

    def call(self, query, values):
        # query hidden state shape == (batch_size, hidden size)
        # query_with_time_axis shape == (batch_size, 1, hidden size)
        # values shape == (batch_size, max_len, hidden size)
        query_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_len, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_len, units)
        score = self.V(tf.nn.tanh(
            self.W1(query_with_time_axis) + self.W2(values)))

        # attention_weights shape == (batch_size, max_len, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import Layer, Conv1D, Softmax, Dense

class SelfAttention(Layer):
    def __init__(self, channels):
        super(SelfAttention, self).__init__()
        # Divide channels for query and key
        self.query = Dense(channels // 8, use_bias=False, kernel_initializer='he_normal')
        self.key = Dense(channels // 8, use_bias=False, kernel_initializer='he_normal')
        self.value = Dense(channels, use_bias=False, kernel_initializer='he_normal')
        
        # Initialize gamma as a trainable parameter
        self.gamma = self.add_weight(name='gamma', shape=[1], initializer=Constant(0.0), trainable=True)

    def call(self, x):
        # Shape of x is (batch, width, channels)
        # Learn query, key, and value vectors
        f = self.query(x)  # Shape (batch, width, channels/8)
        g = self.key(x)    # Shape (batch, width, channels/8)
        h = self.value(x)  # Shape (batch, width, channels)
        
        # Transpose and multiply to get the attention scores
        s = tf.matmul(f, g, transpose_b=True)  # Shape (batch, width, width)
        beta = Softmax(axis=-1)(s)  # Softmax over last dimension to get attention weights
        
        # Apply attention weights to value vector
        o = tf.matmul(beta, h)  # Shape (batch, width, channels)
        # Apply gamma and add input (residual connection)
        return self.gamma * o + x

In [17]:
class TextClassifierHyperModel(HyperModel):
    def __init__(self, input_shape, embeddings_GloVe, num_classes, parallel_blocks):#, include_attention_weights=False):
        self.input_shape = input_shape
        self.embeddings_GloVe = embeddings_GloVe
        self.num_classes = num_classes
        self.parallel_blocks = parallel_blocks
        #self.include_attention_weights = include_attention_weights


    def build(self, hp):
        sequence_input = Input(shape=(self.input_shape,), dtype='int32')
        embedded_sequences = Embedding(input_dim=self.embeddings_GloVe.shape[0],
                                       output_dim=self.embeddings_GloVe.shape[1],
                                       weights=[self.embeddings_GloVe],
                                       trainable=False)(sequence_input)
                                    #    embeddings_regularizer=keras.regularizers.l2(hp.Float('L2_rate', 
                                    #                              min_value=0.0, 
                                    #                              max_value=0.2, 
                                    #                             step=0.3))
                                       
            

        conv_blocks = []
        lstm_blocks = []

        for _ in range (self.parallel_blocks): #(hp.Int('blocks', 1, self.parallel_blocks)):
            conv = Conv1D(
                filters=hp.Int('conv_filter_units', min_value=30, max_value=32, step=1),
                kernel_size= 1, #hp.Int('conv_kernel_size', min_value=1, max_value=3, step=1),
                activation='relu',
                padding='same',
                strides=1)(embedded_sequences)
                
                                                                                            
            # Apply a TimeDistributed Dense layer to each timestep of the convolutional block's output
            conv_dense = TimeDistributed(Dense(hp.Int('lstm_dense_units', min_value=114, max_value=125, step=1), activation='relu'))(conv)
            conv_blocks.append(conv_dense)

            lstm = Bidirectional(LSTM(
                units=hp.Int('lstm_units', min_value=100, max_value=110, step=1),
                return_sequences=True,
                dropout=hp.Float('lstm_dropout_rate', min_value=0.0, max_value=0.04, step=0.01),
                recurrent_dropout=hp.Float('lstm_r_dropout_rate', min_value=0.01, max_value=0.06, step=0.01)
            ))(conv_dense)  # Pass the output of the Dense layer into LSTM
            lstm_blocks.append(lstm)

        combined = Concatenate()(conv_blocks + lstm_blocks)
        # Apply the custom attention mechanism
        attention_layer = AdditiveAttention(use_scale=True)
        attention_output = attention_layer([combined, combined], return_attention_scores=False)
        context_vector = GlobalAveragePooling1D()(attention_output)

        dense = Dense(units=hp.Int('dense_units', min_value=142, max_value=152, step=1), activation='relu')(context_vector)
        dropout = Dropout(0.1)(dense) #Dropout(hp.Float('dropout_rate', min_value=0.0, max_value=0.2, step=0.1))
        outdata = Dense(self.num_classes, activation='sigmoid')(dropout)
        output = (outdata)
        model = Model(inputs=sequence_input, outputs=output)
        
        learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
        optimizer = Adam(learning_rate=learning_rate)
        model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
        return model

# Note: When actually using this model for training in tuner, make sure to remove attention_weights from outputs.
# Hyperparameters and settings
input_length = 20
num_classes = 1
parallel_blocks = 2
log_dir = 'logs/fit/' + datetime.now().strftime("%d-%m-%Y %H-%M-%S")

# Use this for training (without attention weights in outputs)
hypermodel_for_training = TextClassifierHyperModel(input_length, embeddings_GloVe, num_classes, parallel_blocks)#, include_attention_weights=False)


# # Use this for visualization (with attention weights in outputs)
# hypermodel_for_visualization = TextClassifierHyperModel(input_length, embeddings_GloVe, num_classes, parallel_blocks, include_attention_weights=True)

tuner = BayesianOptimization(
    hypermodel_for_training,
    objective='val_accuracy',
    max_trials=10,  # Set the maximum number of trials (model configurations to test)
    executions_per_trial=1,  # Number of models that should be built and fit for each trial
    directory=log_dir,
    project_name='TextClassification'
)

tensorboard_callback = TensorBoard(
    log_dir=log_dir,
    histogram_freq=0,  # No histogram computation, set to 1 or higher to compute histograms every '1' epoch or specified frequency
    update_freq='epoch'  # Log metrics and histograms every epoch (default), not every batch
)

# Setup the checkpoint location
checkpoint_filepath = '/kaggle/working/best_model.h5'

# Create a ModelCheckpoint callback
model_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True
)


# Assume X_train, y_train, X_val, y_val are defined
tuner.search(X_train, y_train, 
             epochs=5, 
             validation_data=(X_test, y_test),
             callbacks=[tensorboard_callback, model_callback])

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
31                |31                |conv_filter_units
115               |115               |lstm_dense_units
101               |101               |lstm_units
0.02              |0.02              |lstm_dropout_rate
0.01              |0.01              |lstm_r_dropout_rate
144               |144               |dense_units
0.0083579         |0.0083579         |learning_rate

Epoch 1/5




In [ ]:
best_model.save(r'C:\Users\bugat\Prosjekter\Tekstanalyse\git_NLP\Tekstanalyse\models\Beast_model')

INFO:tensorflow:Assets written to: C:\Users\bugat\Prosjekter\Tekstanalyse\git_NLP\Tekstanalyse\models\Beast_model\assets


INFO:tensorflow:Assets written to: C:\Users\bugat\Prosjekter\Tekstanalyse\git_NLP\Tekstanalyse\models\Beast_model\assets


In [ ]:
best_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 20)]                 0         []                            
                                                                                                  
 embedding (Embedding)       (None, 20, 100)              2223500   ['input_1[0][0]']             
                                                                                                  
 conv1d (Conv1D)             (None, 20, 31)               3131      ['embedding[0][0]']           
                                                                                                  
 conv1d_1 (Conv1D)           (None, 20, 31)               3131      ['embedding[0][0]']           
                                                                                              

In [ ]:
n_splits=5

KF = KFold(n_splits=n_splits, shuffle=True, random_state=42) # Example: 5-fold cross-validation

# Prepare arrays to store results for each fold
fold_no = 1
loss_per_fold = []
acc_per_fold = []

for train, test in KF.split(X_train_LSTM, y_train_LSTM):
    # Create a fresh model for each fold

    # Fit the model
    best_model.fit(X_train_LSTM[train], y_train_LSTM[train],
                        epochs=n_splits,  # Adjust based on your needs
                        batch_size=64,
                        callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)],
                        validation_data=(X_train_LSTM[test], y_train_LSTM[test]),
                        verbose=1)  # You can set verbose to 0 to reduce logs

    # Evaluate the model
    scores = best_model.evaluate(X_train_LSTM[test], y_train_LSTM[test], verbose=0)
    print(f'Score for fold {fold_no}: {best_model.metrics_names[0]} of {scores[0]}; {best_model.metrics_names[1]} of {scores[1]*100}%')
    loss_per_fold.append(scores[0])
    acc_per_fold.append(scores[1] * 100)
    fold_no += 1

# Print average scores
print(f'Average scores for all folds:\n> Loss: {np.mean(loss_per_fold)}; Accuracy: {np.mean(acc_per_fold)}%')